In [1]:
import sys
sys.path.append('..')
from src.POMDP import *

In [2]:
import numpy as np
import random

# POMDP Toy Problem

Symmetric 3 state, 2 action model

## Model definition

In [3]:
LEFT = -1
RIGHT = 1
ACTIONS = [LEFT, RIGHT]
ACTION_INDEX = {a:a_i for a_i, a in enumerate(ACTIONS)}
ACTION_COUNT = len(ACTIONS)

STATES = [0,1,2]
STATE_COUNT = len(STATES)

In [4]:
# 80% chance of success, 10% land in other tile, 10% stay in place
MOVE_RND = 0.8
def transition_function(s, a, s_p):
    s_a = (s + a) % STATE_COUNT
    s_a = s_a if s_a >= 0 else s_a + STATE_COUNT
    
    if s_a == s_p:
        return MOVE_RND
    else:
        return round((1.0 - MOVE_RND) / (STATE_COUNT - 1), 1)

# Setup tanle to speedup computation
transition_table = np.zeros(shape=(STATE_COUNT, len(ACTIONS), STATE_COUNT))
for s in STATES:
    for a_i, a in enumerate(ACTIONS):
        for s_p in STATES:
            transition_table[s,a_i,s_p] = transition_function(s,a,s_p)

# Randomized transition
def transition(s,a):
    a_i = ACTION_INDEX[a]
    s_p_probs = [transition_table[s, a_i, s_p] for s_p in STATES]
    return np.argmax(np.random.multinomial(n=1, pvals=s_p_probs))

# Test with action 0, ie: LEFT
transition_table[:,0,:]

array([[0.1, 0.1, 0.8],
       [0.8, 0.1, 0.1],
       [0.1, 0.8, 0.1]])

In [5]:
OBS_RND = 0.7 # 70% of observation success
def observation_function(s, a, o):
    a_i = ACTION_INDEX[a]
    # summer = 0
    # for s_p in STATES:
    #     summer += transition_table[s,a_i,s_p] * (OBS_RND if s_p == o else (1.0 - OBS_RND) / (STATE_COUNT - 1))
    # return summer
    return OBS_RND if s == o else (1.0 - OBS_RND) / (STATE_COUNT - 1)

observation_table = np.zeros((STATE_COUNT, ACTION_COUNT, STATE_COUNT))
for s in STATES:
    for a_i, a in enumerate(ACTIONS):
        for o in STATES:
            observation_table[s, a_i, o] = observation_function(s, a, o)

# Random observation 
def observe(s):
    return s if random.random() < OBS_RND else random.choice(list(set(STATES) - set([s])))

# Taking action RIGHT
observation_table[:,0,:]

array([[0.7 , 0.15, 0.15],
       [0.15, 0.7 , 0.15],
       [0.15, 0.15, 0.7 ]])

In [6]:
REWARD_PROBS = [0.2, 0.6, 0.3]
def reward(s):
    return 1 if random.random() < REWARD_PROBS[s] else 0

def reward_function(s, a):
    tot = 0.0
    for s_p in STATES:
        a_i = ACTION_INDEX[a]
        tot += (REWARD_PROBS[s_p] * transition_table[s,a_i,s_p])
    return tot

def immediate_rewards(a):
    return [reward_function(s,a) for s in STATES]

reward_table = np.zeros((STATE_COUNT, len(ACTIONS)))
for s in STATES:
    for a_i,a in enumerate(ACTIONS):
        reward_table[s,a_i] = reward_function(s,a)
        
reward_table

array([[0.32, 0.53],
       [0.25, 0.32],
       [0.53, 0.25]])

In [7]:
for a in ACTIONS:
    for s in STATES:
        print(f'R({s}, {"LEFT" if a == LEFT else "RIGHT"}) = {reward_function(s,a)}')

R(0, LEFT) = 0.32
R(1, LEFT) = 0.25
R(2, LEFT) = 0.53
R(0, RIGHT) = 0.53
R(1, RIGHT) = 0.32
R(2, RIGHT) = 0.25


In [9]:
states = ['s0','s1','s2']
actions = ['left', 'right']

model = POMDPModel(states, actions, transition_table, reward_table, observation_table)

## Solving

In [ ]:
solver = PBVI(model)

In [ ]:
value_function = solver.solve(8,4)